In [1]:
import os
# 【硬性要求】加载环境变量 (确保可以调用 OpenAI API)
from dotenv import load_dotenv
load_dotenv(override=True)

from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

# 导入预构建链工厂函数和 FAISS 向量库
from langchain_classic.chains.combine_documents import create_stuff_documents_chain
from langchain_classic.chains import create_retrieval_chain
from langchain_community.vectorstores import FAISS

# ==========================================
# 1. 数据摄取与索引 (Data Ingestion & Indexing)
# ==========================================
# 依然是那两篇新闻：一篇关于 Anthropic Claude 3，一篇关于 AI21 Labs
URL1 = "https://techcrunch.com/2024/03/04/anthropic-claims-its-new-models-beat-gpt-4/"
URL2 = "https://techcrunch.com/2024/03/28/ai21-labs-new-text-generating-ai-model-is-more-efficient-than-most/"

print("1. 正在抓取网页并切块...")
loader = WebBaseLoader([URL1, URL2])
data = loader.load()

# 文本分块
text_splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=50)
chunks = text_splitter.split_documents(data)

print("2. 正在进行向量化并存入 FAISS 内存数据库...")
# 使用 FAISS 替代 Chroma 进行内存级快速索引
embeddings = OpenAIEmbeddings(model="text-embedding-3-large")
vector = FAISS.from_documents(chunks, embeddings)
retriever = vector.as_retriever() # 转化为检索器组件

# ==========================================
# 2. 构建预构建链 (Building Pre-built Chains)
# ==========================================
print("3. 正在组装高级检索链...")
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.0) # 温度设为0防止模型自行发挥

# 定义提示词。注意：预构建链默认期望用户的问题变量名为 {input}，上下文变量名为 {context}
prompt_template = """
Answer the question {input} based solely on the context below:
\n\n<context>\n{context}\n</context>
If you can't find an answer, say I don't know.
"""
prompt = PromptTemplate.from_template(prompt_template)

# 步骤 A: 创建文档处理内部链 (负责生成答案)
combine_docs_chain = create_stuff_documents_chain(llm, prompt)

# 步骤 B: 创建外层检索链 (负责先检索，再将结果传给步骤 A)
# 这完美替代了我们之前写的 {"context": retriever | format_docs, "input": RunnablePassthrough()}
rag_chain = create_retrieval_chain(retriever, combine_docs_chain)

# ==========================================
# 3. 执行提问 (Execution)
# ==========================================
if __name__ == "__main__":
    # 提问：仅根据提供的上下文，列出 Anthropic 和 Meta 模型及其 Token 大小
    question = "List the models and their token size of models only from Anthropic and Meta"
    print(f"\n🙋 提问: {question}")
    
    # 预构建的检索链 invoke 时，会自动返回一个包含多个键的字典，
    # 真正的答案文本在 'answer' 键中
    result = rag_chain.invoke({"input": question})
    
    print("\n🤖 最终回答:")
    print(result['answer']) # 输出预期：Claude 3 是 200,000，Llama 2 是 32,000

USER_AGENT environment variable not set, consider setting it to identify your requests.


1. 正在抓取网页并切块...
2. 正在进行向量化并存入 FAISS 内存数据库...
3. 正在组装高级检索链...

🙋 提问: List the models and their token size of models only from Anthropic and Meta

🤖 最终回答:
From the context provided, the models and their token sizes are:

1. Anthropic's Claude 3:
   - Initial support for a 200,000-token context window
   - Select customers getting up to a 1-million-token context window (~700,000 words)

2. Meta's Llama 2:
   - ~4,000-token context window

I don't know about the token size of models from Anthropic and Meta other than the ones mentioned above.
